In [1]:
from config import *
import utilities as utils

import importlib

# importlib.reload(utils)

year_files_dict = utils.get_all_filenames()

In [2]:
import gensim
import nltk
import pandas as pd
import re
import itertools

/user/ms5941/.local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
stopwords_list = nltk.corpus.stopwords.words('english')

In [4]:
import json

### Get all tokenized sentences

In [ ]:
all_words = list()
tokenized_doc = dict()

for fname in itertools.chain.from_iterable(year_files_dict.values()):
    tokenized_sentences = list()
    data = utils.load_data(fname)
    count = 0
    for art in data['Items']:
        text = ''
        if art['data']['language'] == 'en':
            if any(subject in art['data']['subjects'] for subject in [US_CODE]):
                text = art['data']['body']

        if not text:
            continue
        count += 1

        text = nltk.sent_tokenize(text)
        text = [sentence.replace('\n', ' ') for sentence in text]
        doc_words = []
        for sentence in text:
            tokenized_sent = list()
            # Only keeping alphabets and replacing everything else with spaces
            sentence = re.sub(r'[^A-Za-z]+', ' ', sentence)
            sentence = sentence.lower()
            sent_words = nltk.word_tokenize(sentence)
            sent_words = [word for word in sent_words if ((len(word) > 1) and (len(word) < 20))]
            sent_words = [word for word in sent_words if word not in stopwords_list]
            for word in sent_words:
                doc_words.append(word)
                all_words.append(word)
                tokenized_sent.append(word)
            tokenized_sentences.append(tokenized_sent)

        tokenized_doc[art['data']['id']] = doc_words
    print(fname, 'Done! Article Count:', count, 'Storing in a json now')
    with open(TEMP_PATH + '/%s.txt' % fname[-15:-9], 'w') as outfile:
        json.dump(tokenized_sentences, outfile)
    del data

/data/ThomsonReuters_NewsArchive/2003/News.RTRS.200301.0214.txt Done! Article Count: 33209 Storing in a json now
/data/ThomsonReuters_NewsArchive/2003/News.RTRS.200308.0214.txt Done! Article Count: 35513 Storing in a json now
/data/ThomsonReuters_NewsArchive/2003/News.RTRS.200312.0214.txt Done! Article Count: 39209 Storing in a json now
/data/ThomsonReuters_NewsArchive/2003/News.RTRS.200302.0214.txt Done! Article Count: 33894 Storing in a json now
/data/ThomsonReuters_NewsArchive/2003/News.RTRS.200307.0214.txt Done! Article Count: 41700 Storing in a json now
/data/ThomsonReuters_NewsArchive/2003/News.RTRS.200305.0214.txt Done! Article Count: 36994 Storing in a json now
/data/ThomsonReuters_NewsArchive/2003/News.RTRS.200306.0214.txt Done! Article Count: 37292 Storing in a json now
/data/ThomsonReuters_NewsArchive/2003/News.RTRS.200310.0214.txt Done! Article Count: 42949 Storing in a json now
/data/ThomsonReuters_NewsArchive/2003/News.RTRS.200303.0214.txt Done! Article Count: 39105 Stori

### Gensim Model - To get the similar words to our three themes

In [ ]:
%env PYTHONHASHSEED = 1
model = Word2Vec(tokenized_sentences, size=100, seed=1, window=4, min_count=2, iter=20)
model.save("all_reuters_news_articles.w2v")

In [ ]:
# TODO
# IF we are going down this path, parallel processing on the grid is probably the best option.
# Or increasing memory allocation for notebook might help too. 